In [2]:
# %load testgpu.py
import mxnet as mx
from mxnet import gluon, init, nd, autograd
from mxnet.gluon import data as gdata, nn, loss as gloss, utils as gutils
import os
#os.environ['CUDA_VISIBLE_DEVICES'] = '1'
import sys
import time
from mxnet.initializer import Xavier
from mxnet.gluon import data as gdata, loss as gloss, model_zoo, nn
def try_gpu():
    """If GPU is available, return mx.gpu(0); else return mx.cpu()."""
    try:
        ctx = mx.gpu()
        _ = nd.array([0], ctx=ctx)
    except mx.base.MXNetError:
        ctx = mx.cpu()
    return ctx

def train_ch5(net, train_iter, test_iter, batch_size, trainer, ctx,
              num_epochs):
    """Train and evaluate a model with CPU or GPU."""
    print('training on', ctx)
    loss = gloss.SoftmaxCrossEntropyLoss()
    for epoch in range(num_epochs):
        train_l_sum, train_acc_sum, n, start = 0.0, 0.0, 0, time.time()
        for X, y in train_iter:
            X, y = X.as_in_context(ctx), y.as_in_context(ctx)
            X = X.astype('float32')
            X=mx.nd.moveaxis(X,-1,1)
            #print(X.shape)
            with autograd.record():
                y_hat = net(X)            
                l = loss(y_hat, y).sum()
            l.backward()
            trainer.step(batch_size)
            y = y.astype('float32')
            train_l_sum += l.asscalar()
            train_acc_sum += (y_hat.argmax(axis=1) == y).sum().asscalar()
            n += y.size
        test_acc = evaluate_accuracy(test_iter, net, ctx)
        print('epoch %d, loss %.4f, train acc %.3f, test acc %.3f, '
              'time %.1f sec'
              % (epoch + 1, train_l_sum / n, train_acc_sum / n, test_acc,
                 time.time() - start))
        
def evaluate_accuracy(data_iter, net, ctx=[mx.cpu()]):
    """Evaluate accuracy of a model on the given data set."""
    if isinstance(ctx, mx.Context):
        ctx = [ctx]
    acc_sum, n = nd.array([0]), 0
    for batch in data_iter:
        features, labels, _ = _get_batch(batch, ctx)
        for X, y in zip(features, labels):
            y = y.astype('float32')
            X = X.astype('float32')
            X=mx.nd.moveaxis(X,-1,1)
            #print(net(X).argmax(axis=1),y)
            acc_sum += (net(X).argmax(axis=1) == y).sum().copyto(mx.cpu())
            n += y.size
        acc_sum.wait_to_read()
    return acc_sum.asscalar() / n

def _get_batch(batch, ctx):
    """Return features and labels on ctx."""
    features, labels = batch
    if labels.dtype != features.dtype:
        labels = labels.astype(features.dtype)
    return (gutils.split_and_load(features, ctx),
            gutils.split_and_load(labels, ctx), features.shape[0])

def load_data_fashion_mnist(batch_size):
    num_workers = 0 if sys.platform.startswith('win32') else 4
    data=gdata.vision.ImageRecordDataset("dataset2.rec")
    train_iter = gdata.DataLoader(data
       , batch_size, shuffle=True,
        num_workers=num_workers)
    data=gdata.vision.ImageRecordDataset("test2.rec")
    test_iter = gdata.DataLoader(data
        , batch_size, shuffle=False,
        num_workers=num_workers)
    return train_iter, test_iter

lr, num_epochs, ctx = 0.001, 50, try_gpu()
batch_size=16


def get_net(ctx):
    resnet = model_zoo.vision.resnet50_v2(pretrained=True)
    resnet.output = nn.Dense(6)
    resnet.output.initialize(init.Xavier(),ctx=ctx)  #for fintune
    resnet.collect_params().reset_ctx(ctx)           #for whole net
    return resnet 
net = get_net(ctx)
net.initialize(init=init.Orthogonal(),ctx=ctx)
net.hybridize()
x = nd.random.uniform(shape=(batch_size,3,256,256), ctx=ctx)

#train_iter, test_iter=load_data_fashion_mnist(batch_size)
trainer = gluon.Trainer(net.collect_params(), 'sgd', {'learning_rate': lr})
train_iter, test_iter=load_data_fashion_mnist(batch_size)
train_ch5(net, train_iter, test_iter, batch_size, trainer, ctx,num_epochs)



/home/park/deeptest/lib/python3.6/site-packages/mxnet/gluon/parameter.py:862: UserWarning: Parameter 'resnetv21_batchnorm0_gamma' is already initialized, ignoring. Set force_reinit=True to re-initialize.
  v.initialize(None, ctx, init, force_reinit=force_reinit)
/home/park/deeptest/lib/python3.6/site-packages/mxnet/gluon/parameter.py:862: UserWarning: Parameter 'resnetv21_batchnorm0_beta' is already initialized, ignoring. Set force_reinit=True to re-initialize.
  v.initialize(None, ctx, init, force_reinit=force_reinit)
/home/park/deeptest/lib/python3.6/site-packages/mxnet/gluon/parameter.py:862: UserWarning: Parameter 'resnetv21_batchnorm0_running_mean' is already initialized, ignoring. Set force_reinit=True to re-initialize.
  v.initialize(None, ctx, init, force_reinit=force_reinit)
/home/park/deeptest/lib/python3.6/site-packages/mxnet/gluon/parameter.py:862: UserWarning: Parameter 'resnetv21_batchnorm0_running_var' is already initialized, ignoring. Set force_reinit=True to re-initial

training on cpu(0)


Process ForkPoolWorker-25:
Process ForkPoolWorker-26:
Process ForkPoolWorker-28:
Traceback (most recent call last):
  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
Traceback (most recent call last):
  File "/usr/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
Traceback (most recent call last):
  File "/usr/lib/python3.6/multiprocessing/pool.py", line 108, in worker
    task = get()
  File "/usr/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python3.6/multiprocessing/queues.py", line 334, in get
    with self._rlock:
  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/usr/lib/python3.6/multiprocessing/pool.py", line 108, in worker
    task = get()
  File "/usr/lib/py

KeyboardInterrupt: 

In [2]:
 net.export("hand_pose_detection",epoch=50)
#mx.model.save(net, "hand_pose_detection", 1000)
#model.save_checkpoint('test',num_epoch) # 保存模型 

In [4]:
#load_model
deserialized_net = gluon.nn.SymbolBlock.imports("hand_pose_detection-symbol.json", ['data'], "hand_pose_detection-0500.params", ctx=ctx)
print(dir(deserialized_net))
train_iter, test_iter=load_data_fashion_mnist(batch_size)
test_acc = evaluate_accuracy(test_iter,deserialized_net, ctx)
print(test_acc)


/home/park/deeptest/lib/python3.6/site-packages/mxnet/gluon/block.py:1159: UserWarning: Cannot decide type for the following arguments. Consider providing them as input:
	data: None
  input_sym_arg_type = in_param.infer_type()[0]


['__call__', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_active', '_alias', '_build_cache', '_cached_graph', '_cached_op', '_call_cached_op', '_check_container_with_block', '_children', '_clear_cached_op', '_collect_params_with_prefix', '_deferred_infer_shape', '_empty_prefix', '_flags', '_forward_hooks', '_forward_pre_hooks', '_get_graph', '_in_format', '_infer_attrs', '_name', '_out_format', '_params', '_prefix', '_reg_params', '_scope', 'apply', 'cast', 'collect_params', 'export', 'forward', 'hybrid_forward', 'hybridize', 'imports', 'infer_shape', 'infer_type', 'initialize', 'load_parameters', 'load_params', 'name', 'name_scope', 'params', 'prefix', 'register_child', 'register_forward_h